In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [2]:
class Customer:
    def __init__(self, customer_id, location, meal_plan, customizations):
        self.customer_id = customer_id
        self.location = location
        self.meal_plan = meal_plan
        self.customizations = customizations

In [3]:
class Vendor:
    def __init__(self, vendor_id, location, cuisine, pricing, capacity):
        self.vendor_id = vendor_id
        self.location = location
        self.cuisine = cuisine
        self.pricing = pricing
        self.capacity = capacity

In [4]:
class Order:
    def __init__(self, order_id, customer_id, vendor_id, items, delivery_time):
        self.order_id = order_id
        self.customer_id = customer_id
        self.vendor_id = vendor_id
        self.items = items
        self.delivery_time = delivery_time

In [5]:
class Rider:
    def __init__(self, rider_id, location):
        self.rider_id = rider_id
        self.location = location
        self.orders = []
        self.route = None

In [6]:
def calculate_distance(loc1, loc2):
    return np.linalg.norm(np.array(loc1) - np.array(loc2))

In [7]:
def assign_orders(orders, vendors):
    for order in orders:
        available_vendors = [v for v in vendors if v.capacity > 0]
        if not available_vendors:
            continue
        nearest_vendor = min(available_vendors, key=lambda v: calculate_distance(v.location, order.customer_id.location))
        nearest_vendor.capacity -= 1
        order.vendor_id = nearest_vendor.vendor_id

In [8]:
def optimize_routes(orders, riders):
    for rider in riders:
        if rider.orders:
            waypoints = [order.customer_location for order in rider.orders]
            directions_result = gmaps.directions(
                origin=rider.location,
                destination=waypoints[-1],
                waypoints=waypoints[:-1],
                optimize_waypoints=True
            )
            if directions_result:
                rider.route = directions_result
                for order in rider.orders:
                    order.delivery_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            else:
                print("Error: No directions found")


In [9]:
def calculate_costs(orders, vendors, delivery_cost_per_km, food_cost_per_meal):
    total_cost = 0
    for order in orders:
        vendor = next(v for v in vendors if v.vendor_id == order.vendor_id)
        distance_to_customer = calculate_distance(vendor.location, order.customer_id.location)
        delivery_cost = distance_to_customer * delivery_cost_per_km
        food_cost = food_cost_per_meal
        total_cost += delivery_cost + food_cost
    return total_cost

In [ ]:
def main():
    # Example data
    customers = [
        Customer(1, (28.7041, 77.1025), '30 days', {'customization': 'vegan'}),
        Customer(2, (28.5355, 77.3910), '15 days', {'customization': 'gluten-free'})
    ]
    vendors = [
        Vendor(1, (28.7041, 77.1025), 'Indian', 100, 10),
        Vendor(2, (28.5355, 77.3910), 'Chinese', 120, 8)
    ]
    orders = [
        Order(1, customers[0], None, ['meal1'], '2024-08-12 12:00:00'),
        Order(2, customers[1], None, ['meal2'], '2024-08-12 13:00:00')
    ]

    delivery_cost_per_km = 2.5
    food_cost_per_meal = 50

    assign_orders(orders, vendors)

    riders = [
        Rider(1, (28.7041, 77.1025)),
        Rider(2, (28.5355, 77.3910))
    ]

    # Assign orders to riders
    for i, order in enumerate(orders):
        riders[i % len(riders)].orders.append(order)

    optimize_routes(orders, riders)
    total_cost = calculate_costs(orders, vendors, delivery_cost_per_km, food_cost_per_meal)

    print(f"Total Cost: Rs. {total_cost:.2f}")

if __name__ == "__main__":
    main()
